# 线性统计模型-数据分析报告处理过程代码MarkDown文档
### 一、数据介绍

#### 1.1 数据来源

数据来自阿里云天池大赛练习赛题，赛题以二手车市场为背景，要求选手预测二手汽车的交易价格，这是一个典型的回归问题，刚好可以用于本次线性统计模型的课程报告。

#### 1.2 数据字段介绍
|     **Field**     |                       **Description**                        |
| :---------------: | :----------------------------------------------------------: |
|      SaleID       |                       交易ID，唯一编码                       |
|       name        |                     汽车交易名称，已脱敏                     |
|      regDate      |          汽车注册日期，例如20160101，2016年01月01日          |
|       model       |                       车型编码，已脱敏                       |
|       brand       |                       汽车品牌，已脱敏                       |
|     bodyType      | 车身类型：豪华轿车：0，微型车：1，厢型车：2，大巴车：3，敞篷车：4，双门汽车：5，商务车：6，搅拌车：7 |
|     fuelType      | 燃油类型：汽油：0，柴油：1，液化石油气：2，天然气：3，混合动力：4，其他：5，电动：6 |
|      gearbox      |                   变速箱：手动：0，自动：1                   |
|       power       |                 发动机功率：范围 [ 0, 600 ]                  |
|     kilometer     |                   汽车已行驶公里，单位万km                   |
| notRepairedDamage |              汽车有尚未修复的损坏：是：0，否：1              |
|    regionCode     |                       地区编码，已脱敏                       |
|      seller       |                  销售方：个体：0，非个体：1                  |
|     offerType     |                  报价类型：提供：0，请求：1                  |
|     creatDate     |                 汽车上线时间，即开始售卖时间                 |
|       price       |                  二手车交易价格（预测目标）                  |
|     v系列特征     |             匿名特征，包含v0-14在内15个匿名特征              |

### 二、特征工程

#### 2.1 读取并观察各数据情况

##### 2.1.1 导入工具包

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#导入warnings包，利用过滤器来实现忽略警告语句。
import warnings
warnings.filterwarnings('ignore')

##### 2.1.2 读取数据文件

该数据文件被分为训练集与数据集需要分别读取

In [ ]:
path = 'D:/car/'
Train_data = pd.read_csv(path+'used_car_train_20200313.csv', sep=' ')
Test_data = pd.read_csv(path+'used_car_testB_20200421.csv', sep=' ')

##### 2.1.3 查看数据

In [ ]:
Train_data.info()

可以发现：

- 此训练集数据共有15万各样本，存在30个变量，虽然变量类型仅notRepairedDamage不是数值型，但里面存在用数字表示的类别变量值得注意
- 其中bodyType等四个字段存在缺失值，等待后续处理
- v_0-v_14数据字段采用了脱敏处理，删除了数据具体含义
- price将是我们要预测的因变量

因为仅notRepairedDamage为object数据，故我们需要详细看一下其数据值的具体内容

In [ ]:
Train_data['notRepairedDamage'].value_counts()

通过上面结果可以发现该数据里面存在“-”符号表示的数据，这也是原始数据中常用来表示缺失值的符号，因为后续模型不一定会用上，故在此先不做处理，而是将其替换为nan

In [ ]:
Train_data['notRepairedDamage'].replace('-', np.nan, inplace=True)

查看训练集的缺失情况：

In [ ]:
missing = Train_data.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()
# 将缺失数量显示在条形图上方
x = range(0, len(missing))
for a, b in zip(x, missing):
    plt.text(a, b + 2, b, ha='center', va='bottom')

通过上图可以直观的发现哪些数据存在缺失，由于暂不确定是否会采用这些变量用于建模，故先不对其进行处理，以免造成信息缺失

In [ ]:
pd.set_option('display.max_columns', 9) #设置最大列数
Train_data.describe()

上面结果显示了数据的统计信息，包括样本数、样本均值等，能对数据的一些情况有大概了解。

查看部分数据字段信息：

In [ ]:
Train_data.head()

上面显示了数据集的前五条数据，可以发现存在日期变量，用1，2，3等数字表示的类别变量以及大多数浮点数表示的数据

##### 2.1.4 数据分离

特征分为类别特征和数值型特征，我们需要分别处理

In [ ]:
# 分离label即预测值
Y_train = Train_data['price']


sns.boxplot(np.log(Y_train+1),orient="v")

In [ ]:
# 这里需要人为根据实际含义来区分数据类型
# 数字特征
numeric_features = ['power', 'kilometer', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 
                    'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14' ]
# 类型特征
categorical_features = ['name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage', 'regionCode']

##### 2.1.5数字特征分析

In [ ]:
# 将因变量加入一起分析
numeric_features.append('price')

1）相关性分析

In [ ]:
# 查看各数字特征与因变量之间的相关系数
price_numeric = Train_data[numeric_features]
correlation = price_numeric.corr()
print(correlation['price'])

通过上面结果可以大致观察出因变量与特征之间的关系强弱，在后续可以考虑设置相应的相关系数阈值来进行变量选取，例如选取相关系数在0.5以上的特征用于回归分析

In [ ]:
# 绘制相关系数矩阵热力图
f , ax = plt.subplots(figsize = (7, 7))

from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

plt.title('相关系数矩阵热力图',y=1,size=16)

sns.heatmap(correlation,square = True,  vmax=0.8)

通过上图可以直观的观察各变量之间的相关性强弱，可以大致看出哪些变量间存在多重共线性，例如v_0和v_3可能存在较强的相关性，可以为后续变量选择或是特征降维工作提供一定的选择基础

2）单变量回归与密度图展示

In [ ]:
# # 线性回归关系图
# #numeric_features.remove('price')
# fcols = 6
# frows = len(numeric_features)
# plt.figure(figsize=(5*fcols,4*frows))

# i = 0
# for col in numeric_features:
#     # 绘制回归图
#     i = i+1
#     ax = plt.subplot(frows,fcols,i)
#     sns.regplot(x=col, y='price', data=price_numeric, ax=ax,
#                scatter_kws = {'marker':'.', 's':3, 'alpha':0.3},
#                line_kws = {'color':'k'})
#     plt.xlabel(col)
#     plt.ylabel('price')
    
#     # 绘制分布图
#     i = i+1
#     ax = plt.subplot(frows,fcols,i)
#     sns.distplot(price_numeric[col].dropna())
#     plt.xlabel(col)

##### 2.1.6类别特征分析

In [ ]:
for fea in categorical_features:
    print(fea,end='\t')
    print(Train_data[fea].nunique())

可以发现：name和regionCode的类别过于稀疏，不适宜用于类别特征分析，先将其剔除

In [ ]:
categorical_features.remove('name')
categorical_features.remove('regionCode')

绘制箱线图观察各类别变量分布情况：

In [ ]:
# plt.figure(figsize=(18, 10))
# plt.boxplot(x=Train_data[categorical_features].values,labels=categorical_features)
# plt.show()

In [ ]:
x=Train_data[categorical_features].values
x

### 三、模型拟合

本部分先仅对数值型特征进行拟合分析，观察拟合效果后视情况讨论类别特征

#### 3.1 代码准备

对于该数据的回归模型拟合采用普通最小二乘方法实现，其中包括最小二乘参数估计、拟合优度计算、单参数检验、F检验、回归诊断、结果输出几个部分，各部分代码通过函数封装，代码展示如下：

In [ ]:
import numpy as np
import scipy.stats as sta
import matplotlib.pyplot as plt


def LeastSquareMethod(X, y):
    '''
        最小二乘法实现
        参数：
        X    自变量矩阵
        y    因变量
        返回值：
        Beta_hat     系数估计值
        y_hat        y估计值
        error_hat    残差
    '''

    mat_temp = X.T * X  # .T表示转置
    Beta_hat = mat_temp.I * X.T * y  # .I表示矩阵求逆

    y_hat = X * Beta_hat
    error_hat = y - y_hat

    return Beta_hat, y_hat, error_hat


def GoodnessOfFit(y, error_hat, N, K):
    '''
        拟合优度计算
        参数：
        y          真实的y
        error_hat  残差
        N          样本量
        K          参数个数
        返回值：
        R_square            拟合优度
        Adjusted_R_square   叫做后的拟合优度
    '''

    SSE = sum(np.multiply(error_hat, error_hat))
    SST = sum(np.multiply(y - y.mean(), y - y.mean()))

    R_square = 1 - SSE / SST
    Adjusted_R_square = 1 - (SSE / (N - K)) / (SST / (N - 1))

    return R_square, Adjusted_R_square


def TTest(X, Beta_hat, error_hat, Beta_test):
    '''
        单个参数检验
        参数：
        X           自变量矩阵
        Beta_hat    系数估计值
        error_hat   残差
        Beta_test   用于原假设的Beta值，一般为0向量
        返回值：
        P           各参数检验的P值
    '''

    # 计算Beat_hat的方差
    mat_temp = np.diagonal((X.T * X).I)
    freedom = X.shape[0] - X.shape[1]
    Beta_var_hat = ((error_hat.T * error_hat / freedom) * mat_temp).T

    # 通过t值计算检验的P值
    t = (Beta_hat - Beta_test) / np.sqrt(Beta_var_hat)
    P = 2 * sta.t.sf(t, freedom)

    return Beta_var_hat, P


def FTest(y, error_hat, N, K):
    '''
        F检验
        参数：
        y          真实的y
        error_hat  残差
        N          样本量
        K          参数个数
        返回值：
        P           各参数检验的P值
    '''

    SSE = sum(np.multiply(error_hat, error_hat))
    SST = sum(np.multiply(y - y.mean(), y - y.mean()))
    SSR = SST - SSE

    # 计算自由度
    freedom_SSR = K - 1
    freedom_SSE = N - K

    # 通过F值计算检验的P值
    F = (SSR / freedom_SSR) / (SSE / freedom_SSE)
    P = sta.f.sf(F, freedom_SSR, freedom_SSE)

    return P


def ErrorRatio(y, y_hat):
    '''
        输入：
        y     真实值
        y_hat 估计值
        输出：
        ErrorRatio 误差百分比
    '''
    temp = abs(y - y_hat) / y * 100

    return temp.sum() / len(y)


def RegressionDiagnosis(X, y_hat, error_hat):
    '''
        回归诊断
        参数：
        X           自变量矩阵
        error_hat   残差
        返回值：
    '''
    
    mu = error_hat.mean()
    sigma = error_hat.std()
    # 绘制残差直方图观察残差是否服从正态分布
    if len(error_hat) > 100:
        num_bins = 30  # 直方图柱子的数量
    else:
        num_bins = 10
    n, bins, patches = plt.hist(error_hat, num_bins, density=1, facecolor='blue', alpha=0.5)
    # n, bins, patches = plt.hist(error_hat, density=1)
    y = sta.norm.pdf(bins, mu, sigma)  # 拟合一条最佳正态分布曲线y
    plt.plot(bins, y, 'r--')  # 绘制y的曲线
    plt.xlabel('sepal-length')  # 绘制x轴
    plt.ylabel('Probability')  # 绘制y轴
    plt.title(r'Histogram : $\mu=%.3f$,$\sigma=%.3f$' % (mu, sigma))  # 中文标题 u'xxx'
    plt.show()
    
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    # 绘制qq图
    sta.probplot(np.array(error_hat).reshape(1,len(error_hat))[0], dist="norm", plot=plt)
    plt.title('(a)')
    # 绘制残差与y_hat的散点图
    plt.subplot(1,2,2)
    plt.scatter(np.array(y_hat), np.array(error_hat))
    plt.axhline(0, c="red")
    plt.title('(b)')

    return

def TheResultsShow(X, y):
    N = X.shape[0]
    K = X.shape[1]
    freedom = N - K
    Beta_test = np.mat(np.zeros(K)).T

    Beta_hat, y_hat, error_hat = LeastSquareMethod(X, y)
    R_square, Adjusted_R_square = GoodnessOfFit(y, error_hat, N, K)
    Beta_var_hat, Pt = TTest(X, Beta_hat, error_hat, Beta_test)
    Pf = FTest(y, error_hat, N, K)
    MSE = error_hat.T * error_hat / freedom
    error_ratio = ErrorRatio(y, y_hat)

    # 打印
    print('+{x:-^{y}}+'.format(x='', y=6 + len('Number of obs')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Prob > F')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('R-squared')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Adj R-squared')) +
          '{x:-^{y}}+'.format(x='', y=8 + len('MSE')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('ErrorRatio')))
    print('|{x:^{y}}|'.format(x='Number of obs', y=6 + len('Number of obs')) +
          '{x:^{y}}|'.format(x='Prob > F', y=6 + len('Prob > F')) +
          '{x:^{y}}|'.format(x='R-squared', y=6 + len('R-squared')) +
          '{x:^{y}}|'.format(x='Adj R-squared', y=6 + len('Adj R-squared')) +
          '{x:^{y}}|'.format(x='MSE', y=8 + len('MSE')) +
          '{x:^{y}}|'.format(x='ErrorRatio', y=6 + len('ErrorRatio')))
    print('+{x:-^{y}}+'.format(x='', y=6 + len('Number of obs')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Prob > F')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('R-squared')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Adj R-squared')) +
          '{x:-^{y}}+'.format(x='', y=8 + len('MSE')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('ErrorRatio')))
    print('|{x:^{y}}|'.format(x='%d' % N, y=6 + len('Number of obs')) +
          '{x:^{y}}|'.format(x='%.3f' % Pf, y=6 + len('Prob > F')) +
          '{x:^{y}}|'.format(x='%.3f' % R_square, y=6 + len('R-squared')) +
          '{x:^{y}}|'.format(x='%.3f' % Adjusted_R_square, y=6 + len('Adj R-squared')) +
          '{x:^{y}}|'.format(x='%.3f' % MSE, y=8 + len('MSE')) +
          '{x:^{y}}|'.format(x='%.3f' % error_ratio, y=6 + len('ErrorRatio')))
    print('+{x:-^{y}}+'.format(x='', y=6 + len('Number of obs')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Prob > F')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('R-squared')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Adj R-squared')) +
          '{x:-^{y}}+'.format(x='', y=8 + len('MSE')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('ErrorRatio')))

    print('+{x:-^{y}}+'.format(x='', y=6 + len('Beta_hat')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Beta_hat')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Std. err.')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('P>|t|')))
    print('|{x:^{y}}|'.format(x='', y=6 + len('Beta_hat')) +
          '{x:^{y}}|'.format(x='Beta_hat', y=6 + len('Beta_hat')) +
          '{x:^{y}}|'.format(x='Std. err.', y=6 + len('Std. err.')) +
          '{x:^{y}}|'.format(x='P>|t|', y=6 + len('P>|t|')))
    print('+{x:-^{y}}+'.format(x='', y=6 + len('Beta_hat')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Beta_hat')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('Std. err.')) +
          '{x:-^{y}}+'.format(x='', y=6 + len('P>|t|')))
    for i in range(len(Beta_hat)):
        print('|{x:^{y}}|'.format(x='Beta_%d' % i, y=6 + len('Beta_hat')) +
              '{x:^{y}}|'.format(x='%.3f' % Beta_hat[i], y=6 + len('Beta_hat')) +
              '{x:^{y}}|'.format(x='%.3f' % Beta_var_hat[i], y=6 + len('Std. err.')) +
              '{x:^{y}}|'.format(x='%.3f' % Pt[i], y=6 + len('P>|t|')))
        # %(i+1, Beta_hat1[i], Beta_hat2[i], Beta_hat3[i]))
        print('+{x:-^{y}}+'.format(x='', y=6 + len('Beta_hat')) +
              '{x:-^{y}}+'.format(x='', y=6 + len('Beta_hat')) +
              '{x:-^{y}}+'.format(x='', y=6 + len('Std. err.')) +
              '{x:-^{y}}+'.format(x='', y=6 + len('P>|t|')))

    # 如果数据为2维或3维则绘图展示
    if K == 2:
        plt.scatter(np.array(X[:,1]), np.array(y))
        plt.plot(np.array(X[:,1]), np.array(y_hat), c='red')
        plt.show()
    elif K == 3:
        ax = plt.axes(projection='3d')  # 绘制3d图形
        ax.scatter(np.array(X[:, 1]), np.array(X[:, 2]), np.array(y), c='r')
        arr_X, arr_Y = np.meshgrid(np.array(X[:,1]), np.array(X[:,2]))
        Z = float(Beta_hat[0]) + float(Beta_hat[1])*arr_X + float(Beta_hat[2])*arr_Y
        ax.plot_surface(arr_X, arr_Y, Z, alpha=0.5, color = "g")
        plt.show()
    else:
        print('维数太高无法绘图展示')

    RegressionDiagnosis(X, y_hat, error_hat)

    return

#### 3.2 模型拟合兼特征选取

为避免过拟合，本部分先将把数据集分为两部分，一部分用于训练，一部分用于检验拟合效果，确定最终变量选取后再将对应的所有数据用于模型求解

##### 3.2.1 数据分割与转换

因为所写的参数估计部分代码采用的是矩阵计算，故需要将数据转换为矩阵形式

power变量极端情况严重，选择删除

In [ ]:
# 通过sklearn的函数进行数据分割，将百分之十的数据用于验证
from sklearn.model_selection import train_test_split
price_numeric = price_numeric.drop(['price','power'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(price_numeric, Y_train, test_size=0.1, random_state=0)

In [ ]:
# 通过2.1.5的相关系数分析结果我们先将相关系数大于0.5的变量选出用于分析
cor = correlation['price']
cor = cor.drop('price')
cor = cor.drop('power')

X_train1 = X_train[cor.index[abs(cor.values)>0.5]]
X_test1 = X_test[cor.index[abs(cor.values)>0.5]]

In [ ]:
# 将数据转换成矩阵形式，并生成增广矩阵
X_test1_mat = np.mat(X_test1)
X_ones = np.mat(np.ones(X_test1_mat.shape[0])).T
X_test1_mat = np.append(X_ones, X_test1_mat, axis=1)

X_train1_mat = np.mat(X_train1)
X_ones = np.mat(np.ones(X_train1_mat.shape[0])).T
X_train1_mat = np.append(X_ones, X_train1_mat, axis=1)

# 转换y
y_train_mat = np.mat(y_train).T
y_test_mat = np.mat(y_test).T

##### 3.2.2 模型拟合

采用自己编写普通最小二乘代码进行模型拟合

由于变量尺度的问题，通过简单的MSE进行误差评估显然已经无法满足要求，下面我们采用误差占比的方式进行误差评估
$$
ErrorRatio = \frac{\sum{\frac{|y-\hat{y}|}{y}\times 100}}{N}
$$

In [ ]:
# 考虑暴力添加，直接将所有不含nan的数值型变量加入该模型进行分析
X_train3_mat = np.mat(X_train.dropna(axis=1))
X_ones = np.mat(np.ones(X_train3_mat.shape[0])).T
X_train3_mat = np.append(X_ones, X_train3_mat, axis=1)

TheResultsShow(X_train3_mat, y_train_mat)

通过作图我们发现数据的标签（price）呈现长尾分布，不利于我们的建模预测。原因是很多模型都假设数据误差项符合正态分布，而长尾分布的数据违背了这一假设。

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.distplot(y_train)
plt.title('(a)')
plt.subplot(1,2,2)
sns.distplot(y_train[y_train < np.quantile(y_train, 0.9)])
plt.title('(b)')

In [ ]:
train_y_ln = np.log(y_train + 1)

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.distplot(train_y_ln)
plt.title('(a)')
plt.subplot(1,2,2)
sta.probplot(np.array(train_y_ln).reshape(1,len(train_y_ln))[0], dist="norm", plot=plt)
plt.title('(b)')

In [ ]:
train_y_ln = np.mat(train_y_ln).T

In [ ]:
TheResultsShow(X_train3_mat, train_y_ln)

In [ ]:
sns.boxplot(train_y_ln,orient="v")

通过上面的结果可以发现模型在第三个变量的P值很大，在这里考虑将其删除再拟合一次

In [ ]:
# 添加相关系数大于0.1的变量
X_train4 = X_train[cor.index[abs(cor.values)>0.1]]

X_train4_mat = np.mat(X_train4)
X_ones = np.mat(np.ones(X_train4_mat.shape[0])).T
X_train4_mat = np.append(X_ones, X_train4_mat, axis=1)

TheResultsShow(X_train4_mat, train_y_ln)

In [ ]:
X_train1

In [ ]:
TheResultsShow(X_train1_mat, train_y_ln)

In [ ]:
X_train2_mat = np.delete(X_train1_mat, 2, axis=1)
TheResultsShow(X_train2_mat, train_y_ln)

可以发现删除第二个变量后该模型的其他特征全部通过了检验，但该模型的拟合优度为0.621，相对来说较低，而且我们还有许多变量还未用上，故下面考虑增加变量来进行模型拟合

####  统计预测

In [ ]:
X_train1

In [ ]:
X_train1_mat

In [ ]:
X_train2_mat

In [ ]:
Beta_hat, y_hat, error_hat = LeastSquareMethod(X_train2_mat, train_y_ln)

X_test_mat = X_test[['v_0', 'v_8', 'v_12']]
X_test_mat = np.mat(X_test_mat)
X_ones = np.mat(np.ones(X_test_mat.shape[0])).T
X_test_mat = np.append(X_ones, X_test_mat, axis=1)

In [ ]:
y_pre_ln = X_test_mat * Beta_hat
y_pre_ln

In [ ]:
y_test_mat.shape

In [ ]:
y_pre = np.exp(y_pre_ln)-1
error_hat = y_test_mat - y_pre

RegressionDiagnosis(X_test_mat, y_pre, error_hat)

er = ErrorRatio(y_test_mat, y_pre)
er

In [ ]:
# 查看训练集与测试集的数据分布是否一致
dist_cols = 3
dist_rows = 1
plt.figure(figsize=(4*dist_cols,4*dist_rows))
i = 1
for col in ['v_0', 'v_8', 'v_12']:
    ax = plt.subplot(dist_rows, dist_cols, i)
    ax = sns.kdeplot(X_train[col],color='red',shade=True)
    ax = sns.kdeplot(X_test[col],color='blue',shade=True)
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax = ax.legend(['train', 'test'])
    i = i+1
plt.show()

In [ ]:
sns.boxplot(train_y_ln)

In [ ]:

sns.boxplot(np.log(y_test+1),orient="v")

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.distplot(np.log(y_test+1))
plt.title('(a)')
plt.subplot(1,2,2)
sns.distplot(np.log(y_test+1)[np.log(y_test+1) < np.quantile(np.log(y_test+1), 0.9)])
plt.title('(b)')

In [ ]:
def xxx(X, error_hat):
    # 计算标准化残差
    freedom = X.shape[0] - X.shape[1]
    theta = np.sqrt(error_hat.T * error_hat / freedom)
    Std_error = error_hat / theta

    # 计算学生化残差
    mat_temp = X.T * X
    Hii = np.diagonal(X * mat_temp.I * X.T)
    Stu_error = error_hat / (theta * np.sqrt(1-Hii)).T

    # 计算DFFITS
    MSE = error_hat.T * error_hat / freedom
    DFFITSi = error_hat / np.sqrt(MSE * Hii).T

    # 计算Cook distances
    k = X.shape[1]
    temp1 = np.multiply(error_hat, error_hat) / (k*MSE)
    temp2 = np.mat(Hii / np.multiply(1-Hii, 1-Hii))
    Di = np.multiply(temp1, temp2.T)
    
    return Di

In [ ]:
# 考虑3sigma原则删除异常数据
#设定法则的左右边界
left=y_train.mean()-3*y_train.std()
right=y_train.mean()+3*y_train.std()

#获取在范围内的数据
new_num_y=y_train[(left>y_train)|(y_train>right)]
new_num_y

y_new = y_train.drop(new_num_y.index[:])
y_new = np.mat(y_new)
y_new_ln = np.log(y_new+1).T
y_new_ln

sns.boxplot(y_new_ln,orient="v")

In [ ]:
# 索引数据量太大无法直接截取，所以我们采取删除不满足条件的数据


In [ ]:
X_train_new = X_train.drop(new_num_y.index[:])
X_train_new = X_train_new[['v_0', 'v_8', 'v_12']]

X_train_new = np.mat(X_train_new)
X_ones = np.mat(np.ones(X_train_new.shape[0])).T
X_train_new = np.append(X_ones, X_train_new, axis=1)

In [ ]:
TheResultsShow(X_train_new, y_new_ln)

In [ ]:
Beta_hat, y_hat, error_hat = LeastSquareMethod(X_train_new, y_new_ln)

In [ ]:
y_pre_ln = X_test_mat * Beta_hat
y_pre_ln

y_pre = np.exp(y_pre_ln)-1
error_hat = y_test_mat - y_pre

RegressionDiagnosis(X_test_mat, y_pre, error_hat)

er = ErrorRatio(y_test_mat, y_pre)
er

In [ ]:
# 取四分位数删除异常数据
#设定法则的左右边界
left=y_train.quantile(0.05)
right=y_train.quantile(0.95)

#获取在范围内的数据
new_num_y=y_train[(left>y_train)|(y_train>right)]
new_num_y

y_new = y_train.drop(new_num_y.index[:])
y_new = np.mat(y_new)
y_new_ln = np.log(y_new+1).T
y_new_ln

sns.boxplot(y_new_ln,orient="v")

X_train_new = X_train.drop(new_num_y.index[:])
X_train_new = X_train_new[['v_0', 'v_8', 'v_12']]

X_train_new = np.mat(X_train_new)
X_ones = np.mat(np.ones(X_train_new.shape[0])).T
X_train_new = np.append(X_ones, X_train_new, axis=1)

In [ ]:
TheResultsShow(X_train_new, y_new_ln)

In [ ]:
Beta_hat, y_hat, error_hat = LeastSquareMethod(X_train_new, y_new_ln)

y_pre_ln = X_test_mat * Beta_hat
y_pre_ln

y_pre = np.exp(y_pre_ln)-1
error_hat = y_test_mat - y_pre

RegressionDiagnosis(X_test_mat, y_pre, error_hat)

er = ErrorRatio(y_test_mat, y_pre)
er

In [ ]:
# 考虑3sigma原则删除异常数据
#设定法则的左右边界
left=y_test.quantile(0.25)
right=y_test.quantile(0.75)
# left=y_test.mean()-3*y_test.std()
# right=y_test.mean()+3*y_test.std()

#获取在范围内的数据
new_num_y_test=y_test[(left>y_test)|(y_test>right)]
new_num_y_test


y_new_test = y_test.drop(new_num_y_test.index[:])
y_new_test = np.mat(y_new_test)
y_new_test_ln = np.log(y_new_test+1).T
y_new_test_ln

In [ ]:
X_test_new = X_test.drop(new_num_y_test.index[:])
X_test_new = X_test_new[['v_0', 'v_8', 'v_12']]

X_test_new = np.mat(X_test_new)
X_ones = np.mat(np.ones(X_test_new.shape[0])).T
X_test_new = np.append(X_ones, X_test_new, axis=1)

In [ ]:
y_pre_ln = X_test_new * Beta_hat
y_pre_ln

y_pre = np.exp(y_pre_ln)-1
y_pre
error_hat = y_new_test.T - y_pre

RegressionDiagnosis(X_test_mat, y_pre, error_hat)

er = ErrorRatio(y_new_test.T, y_pre)
er

In [ ]:
error_hat

In [ ]:
sns.boxplot(y_new_test,orient="v")

In [ ]:
# 取四分位数删除异常数据
#设定法则的左右边界
median=y_train.quantile(0.93)

#获取在范围内的数据
new_num_y=y_train[(median<y_train)]
new_num_y

y_new = y_train.drop(new_num_y.index[:])
y_new = np.mat(y_new)
y_new_ln = np.log(y_new+1).T
y_new_ln

sns.boxplot(y_new_ln,orient="v")

X_train_new = X_train.drop(new_num_y.index[:])
X_train_new = X_train_new[['v_0', 'v_8', 'v_12']]

X_train_new = np.mat(X_train_new)
X_ones = np.mat(np.ones(X_train_new.shape[0])).T
X_train_new = np.append(X_ones, X_train_new, axis=1)

In [ ]:
TheResultsShow(X_train_new, y_new_ln)

In [ ]:
#获取在范围内的训练数据
new_num_y_test=y_test[(median<y_test)]
new_num_y_test

y_new_test = y_test.drop(new_num_y_test.index[:])
y_new_test = np.mat(y_new_test)


X_test_new = X_test.drop(new_num_y_test.index[:])
X_test_new = X_test_new[['v_0', 'v_8', 'v_12']]

X_test_new = np.mat(X_test_new)
X_ones = np.mat(np.ones(X_test_new.shape[0])).T
X_test_new = np.append(X_ones, X_test_new, axis=1)

In [ ]:
Beta_hat, y_hat, error_hat = LeastSquareMethod(X_train_new, y_new_ln)

y_pre_ln = X_test_new * Beta_hat
y_pre_ln

y_pre = np.exp(y_pre_ln)-1
y_pre

y_test_mat = np.mat(y_test).T

error_hat = y_new_test.T - y_pre

RegressionDiagnosis(X_test_new, y_pre, error_hat)

er = ErrorRatio(y_new_test.T, y_pre)
er

In [ ]:
X_train.shape

### 加入类别变量

In [ ]:
Train_data1 = Train_data[['bodyType', 'fuelType', 'gearbox', 'v_0', 'v_8', 'v_12', 'price']]
Train_data1 = Train_data1.dropna(axis=0, how='any', inplace=False)
Y_train = Train_data1['price']
X_train = Train_data1[['bodyType', 'fuelType', 'gearbox', 'v_0', 'v_8', 'v_12']]

In [ ]:
#Train_data1 = Train_data[['bodyType', 'fuelType', 'gearbox', 'v_0', 'v_8', 'v_12', 'price']]
Train_data1 = Train_data.dropna(axis=0, how='any', inplace=False)
Y_train = Train_data1['price']
X_train = Train_data1[[ 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'bodyType', 'fuelType', 'gearbox',
                       'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14']]
# numeric_features = ['power', 'kilometer', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 
#                     'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14' ]
# # 类型特征
# categorical_features = ['name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage', 'regionCode']

In [ ]:
correlation = Train_data1.corr()
print(correlation['price'])

In [ ]:
X_train

In [ ]:
# 通过sklearn的函数进行数据分割，将百分之十的数据用于验证
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)

In [ ]:
# 考虑pca降维
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
X_train_09 = pca.fit_transform(X_train)

In [ ]:
X_train_09
X_ones = np.mat(np.ones(X_train_09.shape[0])).T
X_train_09 = np.append(X_ones, X_train_09, axis=1)

In [ ]:
X_train_09

In [ ]:
X_test_09 = pca.transform(X_test)
X_ones = np.mat(np.ones(X_test_09.shape[0])).T
X_test_09 = np.append(X_ones, X_test_09, axis=1)

In [ ]:
# 转换y
y_train_mat = np.mat(y_train).T
y_test_mat = np.mat(y_test).T

train_y_ln = np.log(y_train_mat + 1)

In [ ]:
Beta_hat, y_hat, error_hat = LeastSquareMethod(X_train_09, train_y_ln)

y_pre_ln = X_test_09 * Beta_hat
y_pre_ln

y_pre = np.exp(y_pre_ln)-1
y_pre

y_test_mat = np.mat(y_test).T

error_hat = y_test_mat - y_pre

RegressionDiagnosis(X_test_09, y_pre, error_hat)

er = ErrorRatio(y_test_mat, y_pre)
er

In [ ]:
# 最终采用PCA降维进行回归，导入所有训练数据
Train_data1 = Train_data[['bodyType', 'fuelType', 'gearbox', 'v_0', 'v_8', 'v_12', 'price']]
Train_data1 = Train_data1.dropna(axis=0, how='any', inplace=False)
Y_train = Train_data1['price']
X_train = Train_data1[['bodyType', 'fuelType', 'gearbox', 'v_0', 'v_8', 'v_12']]

In [ ]:
Train_data1 = Train_data.dropna(axis=0, how='any', inplace=False)
Y_train = Train_data1['price']
X_train = Train_data1[[ 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'bodyType', 'fuelType', 'gearbox',
                       'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14']]

In [ ]:
# 考虑pca降维
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
X_train_mat = pca.fit_transform(X_train)
X_ones = np.mat(np.ones(X_train_mat.shape[0])).T
X_train_mat = np.append(X_ones, X_train_mat, axis=1)

In [ ]:
# 转换y
Y_train_mat = np.mat(Y_train).T

train_Y_ln = np.log(Y_train_mat + 1)

In [ ]:
train_Y_ln.shape

In [ ]:
TheResultsShow(X_train_mat, train_Y_ln)

In [ ]:
# 将数据转换成矩阵形式，并生成增广矩阵
X_test1_mat = np.mat(X_test)
X_ones = np.mat(np.ones(X_test1_mat.shape[0])).T
X_test1_mat = np.append(X_ones, X_test1_mat, axis=1)

X_train1_mat = np.mat(X_train)
X_ones = np.mat(np.ones(X_train1_mat.shape[0])).T
X_train1_mat = np.append(X_ones, X_train1_mat, axis=1)



In [ ]:
TheResultsShow(X_train_09, train_y_ln)

In [ ]:
Beta_hat, y_hat, error_hat = LeastSquareMethod(X_train1_mat, train_y_ln)

In [ ]:
y_pre_ln = X_test1_mat * Beta_hat
y_pre_ln

y_pre = np.exp(y_pre_ln)-1
y_pre
error_hat = y_test_mat - y_pre

RegressionDiagnosis(X_test1_mat, y_pre, error_hat)

er = ErrorRatio(y_test_mat, y_pre)
er

In [ ]:
# 考虑数据划分
# 取四分位数删除异常数据
#设定法则的左右边界
median=y_train.quantile(0.93)

#获取在范围内的数据
new_num_y=y_train[(median>y_train)]
new_num_y

y_new = y_train.drop(new_num_y.index[:])
y_new = np.mat(y_new)
y_new_ln = np.log(y_new+1).T
y_new_ln

sns.boxplot(y_new_ln,orient="v")

X_train_new = X_train.drop(new_num_y.index[:])
# 考虑pca降维
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
X_train_new_09 = pca.fit_transform(X_train_new)

In [ ]:
X_ones = np.mat(np.ones(X_train_new_09.shape[0])).T
X_train_new_09 = np.append(X_ones, X_train_new_09, axis=1)

In [ ]:
TheResultsShow(X_train_new_09, y_new_ln)

In [ ]:
#获取在范围内的训练数据
new_num_y_test=y_test[(median>y_test)]
new_num_y_test

y_new_test = y_test.drop(new_num_y_test.index[:])
y_new_test = np.mat(y_new_test)


X_test_new = X_test.drop(new_num_y_test.index[:])

X_test_new_09 = pca.transform(X_test_new)
X_ones = np.mat(np.ones(X_test_new_09.shape[0])).T
X_test_new_09 = np.append(X_ones, X_test_new_09, axis=1)

In [ ]:
Beta_hat, y_hat, error_hat = LeastSquareMethod(X_train_new_09, y_new_ln)

y_pre_ln = X_test_new_09 * Beta_hat
y_pre_ln

y_pre = np.exp(y_pre_ln)-1
y_pre

y_test_mat = np.mat(y_test).T

error_hat = y_new_test.T - y_pre

RegressionDiagnosis(X_test_new_09, y_pre, error_hat)

er = ErrorRatio(y_new_test.T, y_pre)
er